In [2]:
import os

from transformers import CLIPProcessor, CLIPModel

import torch
from torchvision.datasets import ImageNet

from PIL import Image
from tqdm import tqdm

# 设置新的工作目录 
os.chdir('/home/ubuntu/erjia/LLaVA/')
print("change curr dir to", os.getcwd())

device = "cuda" if torch.cuda.is_available() else "cpu"

# 1. 定义ImageNet标签（1000类）
dataset = ImageNet(root='datasets/ImageNet/', split='val')
imagenet_labels = ['a photo of ' + label[0] for label in dataset.classes]

# 2. 加载CLIP模型和处理器
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14-336")
model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14-336")

model.to(device)
device

/home/ubuntu/anaconda3/envs/llava/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2023-10-31 02:00:13,619] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
change curr dir to /home/ubuntu/erjia/LLaVA


`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


'cuda'

In [5]:
# 3. 零样本分类并计算准确率
correct = 0
total = 0

# 遍历文件夹中的所有文件
image_folder = 'images/species-r1'
for filename in tqdm(os.listdir(image_folder)):
    # 检查文件是否为图片
    if filename.endswith(".jpg"):  

        image_path = os.path.join(image_folder, filename)
        
        img = Image.open(image_path)
        
        label = 'a photo of ' + image_path.split('.')[0].split('-')[-2]
        mislabel = 'a photo of ' + image_path.split('.')[0].split('-')[-1]

        with torch.no_grad():
            # 获取文本和图像的特征
            text_inputs = processor(images=None, text=imagenet_labels, return_tensors="pt", padding=True, truncation=True).to(device)
            image_inputs = processor(images=img, text=None, return_tensors="pt").to(device)

            # 获取模型输出
            outputs = model(**text_inputs, **image_inputs)
            image_features = outputs.image_embeds
            text_features = outputs.text_embeds

            # 计算相似性分数
            similarities = (image_features @ text_features.T) / text_features.norm(dim=-1)
            
            # 预测
            pred = torch.argmax(similarities, dim=-1).item()
            correct += (imagenet_labels[pred] == label)
            total += 1
            
accuracy = correct / total
print(f"Zero-shot Accuracy on ImageNet: {accuracy * 100:.2f}%")

100%|██████████| 998/998 [03:34<00:00,  4.64it/s]

Zero-shot Accuracy on ImageNet: 28.26%
